In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('./dataset/AB_NYC_2019.csv')
len(df)

48895

In [3]:
len(df.columns)

16

In [4]:
df.drop(['id', 'calculated_host_listings_count', 'availability_365'], axis = 1, inplace = True)

In [5]:
df.to_csv('./dataset/refine_booking_NYC.csv')


In [6]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

a = df['price'].values.reshape(-1, 1) 

df.loc[:,'price'] = scaler.fit_transform(a)


In [7]:
from sklearn.utils import shuffle
df_shuffle = shuffle(df)

In [8]:
print(len(df_shuffle))
num = 38000
df_tr = df_shuffle[:num] 
df_te = df_shuffle[num:]

48895


In [9]:
df_tr.to_csv('./dataset/train.csv', index=False)
df_te.to_csv('./dataset/test.csv', index=False)

## 使用train.csv建立baseline

In [10]:
df = pd.read_csv('./dataset/train.csv')
len(df)

38000

In [11]:
df.head()

,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month
0,Dekalb Stop Flat,25545946,Tyler,Queens,Ridgewood,40.70772,-73.91180,Private room,-0.302811,2,0,NaN,NaN
1,Amazing apartment Times Square,265068881,Omer,Manhattan,Hell's Kitchen,40.76350,-73.98780,Entire home/apt,-0.032149,4,2,2019-06-26,2.00
2,Brownstone in Clinton Hill,2427529,Drew,Brooklyn,Clinton Hill,40.68409,-73.96264,Private room,-0.323632,1,0,NaN,NaN
3,Beautiful Apartment in the heart of Chelsea,24047286,Sabrina,Manhattan,Chelsea,40.74560,-73.99771,Entire home/apt,0.405074,4,1,2017-10-12,0.05
4,NY HILTON CLUB 2 BR SUITE Luxury Holidays 2019,58071546,Mary,Manhattan,Midtown,40.76151,-73.97894,Entire home/apt,1.858322,2,3,2019-01-03,0.24


In [12]:
df.isnull().any()

name                    True
host_id                False
host_name               True
neighbourhood_group    False
neighbourhood          False
latitude               False
longitude              False
room_type              False
price                  False
minimum_nights         False
number_of_reviews      False
last_review             True
reviews_per_month       True
dtype: bool

In [13]:
df = df.dropna()
df.isnull().any()

name                   False
host_id                False
host_name              False
neighbourhood_group    False
neighbourhood          False
latitude               False
longitude              False
room_type              False
price                  False
minimum_nights         False
number_of_reviews      False
last_review            False
reviews_per_month      False
dtype: bool

In [14]:
len(df)

30175

In [15]:
df.select_dtypes('object').columns

Index(['name', 'host_name', 'neighbourhood_group', 'neighbourhood',
       'room_type', 'last_review'],
      dtype='object')

In [16]:
dfc = df.copy()

In [17]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()

lis = df.select_dtypes('object').columns

for i in lis:
    df[i] = labelencoder.fit_transform(df[i])

df.select_dtypes('object').columns

Index([], dtype='object')

In [18]:
len(set(df['host_id']))

24306

In [19]:
from sklearn.model_selection import train_test_split

y = df.pop('price')

X_train, X_test, y_train, y_test = train_test_split(df, y,
                                                    test_size=0.33, random_state=42)



In [20]:
y_train.shape

(20217,)

In [21]:
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.fit_transform(X_test)

In [22]:
X_train.isnull().any()

name                   False
host_id                False
host_name              False
neighbourhood_group    False
neighbourhood          False
latitude               False
longitude              False
room_type              False
minimum_nights         False
number_of_reviews      False
last_review            False
reviews_per_month      False
dtype: bool

In [23]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
def score(y_true, y_pred):
# y_true = y_train
# y_pred = clf.predict(X_train)
    print('---')
    print('R2, ', r2_score(y_true, y_pred))
    print('mse, ', mse(y_true, y_pred))
    print('mae, ', mae(y_true, y_pred))
    print('rmse, ', mse(y_true, y_pred) ** 0.5)
    print('---')

### 演算法1: Reg問題的通用baseLine model:  Zero-Rule model
他很爛 但很快 很簡單 可以告訴你這批資料的樣態

In [24]:
#Zero Rule base
y_pred = [y_train.mean()]*len(y_train)
print(y_pred[:10],'\n')
score(y_train, y_pred)

[-0.04757077866587187, -0.04757077866587187, -0.04757077866587187, -0.04757077866587187, -0.04757077866587187, -0.04757077866587187, -0.04757077866587187, -0.04757077866587187, -0.04757077866587187, -0.04757077866587187] 

---
R2,  0.0
mse,  0.48667663124951144
mae,  0.32912534341101946
rmse,  0.697622126404769
---


### 演算法2: SVM Reg: 這個要跑很久  

In [25]:
from sklearn.svm import SVR


    
# 定義SVR
svr = SVR(kernel ='rbf', C = 1)

# svr.fit(X_train, y_train)

# score(y_train, svr.predict(X_train))
    

## C值很大的後果(與測試做對照)
Ref
https://scikit-learn.org/stable/auto_examples/svm/plot_rbf_parameters.html

In [26]:
# score(y_train, svr.predict(X_train))
# score(y_test, svr.predict(X_test))

### 演算法3: XGBoost

In [27]:
import xgboost as xgb

def XGB_def():
    xgbreg = xgb.XGBRegressor(
    #樹的個數
    n_estimators=1000,
    # 如同學習率
    learning_rate= 0.3, 
    # 構建樹的深度，越大越容易過擬合    
    max_depth=6, 
    # 隨機取樣訓練樣本 訓練例項的子取樣比
    subsample=1, 
    # 用於控制是否後剪枝的引數,越大越保守，一般0.1、0.2這樣子
    gamma=0, 
    # 控制模型複雜度的權重值的L2正則化項引數，引數越大，模型越不容易過擬合。
    reg_lambda=1,  
    #隨機種子
    seed=1000, 
    # L1 正則項引數
#        reg_alpha=0,
    #設定成1則沒有執行資訊輸出，最好是設定為0.是否在執行升級時列印訊息。
    silent=0 ,
    tree_method = 'gpu_hist', 
    predictor = 'gpu_predictor',
    eval_metric= 'mse',
    objective = 'reg:squarederror'
    )
    
    return xgbreg

# 定義xgb
def XGB_train_score(X_train, y_train):
    # 定義模型
    xgbreg = XGB_def()
    
    # 模型 訓練
    xgbreg.fit(X_train,y_train,eval_metric='rmse')

    # 預測值
    y_pred=xgbreg.predict(X_train)

    # 評估
    score(y_train, y_pred)
    return xgbreg

In [32]:
X_train

,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,minimum_nights,number_of_reviews,last_review,reviews_per_month
24363,8948,22474793,207,1,106,40.64046,-73.97233,0,1,2,1653,1.28
2253,5154,4410830,5384,1,211,40.71162,-73.96431,0,4,4,1681,0.38
29222,8680,5986790,2669,2,203,40.83383,-73.94469,1,7,13,1637,0.36
17279,2918,1307773,3995,2,203,40.85099,-73.92822,1,60,4,272,0.06
12705,12705,2119276,3083,2,93,40.76494,-73.98810,0,30,15,1423,0.29
...,...,...,...,...,...,...,...,...,...,...,...,...
37524,5736,5334697,7009,2,203,40.84099,-73.93694,1,5,41,1678,1.45
6767,17732,6405437,3701,1,211,40.71201,-73.96144,0,2,6,1435,0.25
1074,17286,136431007,4690,2,33,40.74944,-73.99774,1,1,40,1666,1.63
19889,23559,144857346,2821,2,141,40.72485,-73.99461,0,3,1,1141,0.05


In [28]:
XGB_train_score(X_train, y_train)
XGB_train_score(X_test, y_test)

/opt/conda/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


---
R2,  0.9979780477788709
mse,  0.0009840368955265776
mae,  0.02126540085331307
rmse,  0.031369362370417694
---


/opt/conda/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


---
R2,  0.9999729521131525
mse,  2.648380682254907e-05
mae,  0.003475819245453743
rmse,  0.0051462420097143765
---


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, eval_metric='mse', gamma=0,
             importance_type='gain', learning_rate=0.3, max_delta_step=0,
             max_depth=6, min_child_weight=1, missing=None, n_estimators=1000,
             n_jobs=1, nthread=None, objective='reg:squarederror',
             predictor='gpu_predictor', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, seed=1000, silent=0, subsample=1,
             tree_method='gpu_hist', verbosity=1)

### 演算法4: 隨機森林

In [28]:
from sklearn.ensemble import RandomForestRegressor
def RF_def():
    RFreg = RandomForestRegressor(max_depth=4, random_state=0, n_estimators = 1000,
                            criterion = 'mse', n_jobs = 16)
    return RFreg
def RF_train_score(X_train, y_train):
    RFreg = RF_def()
    # 模型 訓練
    RFreg.fit(X_train,y_train)

    # 預測值
    y_pred=RFreg.predict(X_train)

    # 評估
    score(y_train, y_pred)
    return RFreg

In [29]:
RF_train_score(X_train, y_train)

---
R2,  0.3079097529025896
mse,  0.3341645493140877
mae,  0.23411662144871107
rmse,  0.5780696751379436
---


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=4,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=1000,
                      n_jobs=16, oob_score=False, random_state=0, verbose=0,
                      warm_start=False)

### 第一次評估: 
因為 mse遠大於mae，可以認為裡面有很多離群值，接下來對離群值做處理。但其實tree_base的演算法(XGB, CART, RandomFroest)對離群值不太敏感，所以我們等一下的分析包括:
- 離群值
- 變數相關性
- 找出沒用的變數
特徵工程

## Chap2: 尋找離群值

In [30]:
# 把df做還原
df = dfc.copy()
df.head(3)

,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month
0,Beautiful Ft. Greene Apartment-NEW!,2966937,Chris,Brooklyn,Clinton Hill,40.68533,-73.96808,Entire home/apt,-0.015493,2,20,2018-12-30,0.24
1,East Williamsburg/Bushwick Loft,130667097,Paul,Brooklyn,Williamsburg,40.70492,-73.93341,Private room,-0.406912,1,25,2018-12-30,0.96
2,Sunny and comfortable Upper West Side Manhattan,169704873,Michele,Manhattan,Morningside Heights,40.81064,-73.95734,Private room,-0.011329,14,6,2018-06-22,0.41


In [31]:
# 所有不是str(object)的欄位
df_math = df.select_dtypes(exclude=['object'])
df_math.head()

,host_id,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month
0,2966937,40.68533,-73.96808,-0.015493,2,20,0.24
1,130667097,40.70492,-73.93341,-0.406912,1,25,0.96
2,169704873,40.81064,-73.95734,-0.011329,14,6,0.41
3,27403581,40.82665,-73.94669,-0.431897,2,52,1.95
5,3571172,40.81251,-73.94524,-0.240351,1,2,0.06


In [32]:
# host_id也不是我們要分析離群值的目標 把他也拿掉

df_math.drop(columns = ['host_id'], axis = 1, inplace = True)
df_math

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month
0,40.68533,-73.96808,-0.015493,2,20,0.24
1,40.70492,-73.93341,-0.406912,1,25,0.96
2,40.81064,-73.95734,-0.011329,14,6,0.41
3,40.82665,-73.94669,-0.431897,2,52,1.95
5,40.81251,-73.94524,-0.240351,1,2,0.06
...,...,...,...,...,...,...
37993,40.66892,-73.99136,0.051131,3,178,3.70
37994,40.71569,-73.99076,-0.373600,3,10,0.36
37996,40.68591,-73.91344,-0.448553,1,18,0.41
37998,40.83555,-73.87570,-0.531833,3,38,1.82


In [33]:
# 
len(df_math.mask((df_math - df_math.mean()).abs() > 2 * df_math.std()).dropna())

24383

In [34]:
a = df_math.mask((df_math - df_math.mean()).abs() >2 * df_math.std())
a

,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month
0,40.68533,-73.96808,-0.015493,2.0,20.0,0.24
1,40.70492,-73.93341,-0.406912,1.0,25.0,0.96
2,40.81064,-73.95734,-0.011329,14.0,6.0,0.41
3,40.82665,-73.94669,-0.431897,2.0,52.0,1.95
5,40.81251,-73.94524,-0.240351,1.0,2.0,0.06
...,...,...,...,...,...,...
37993,40.66892,-73.99136,0.051131,3.0,NaN,3.70
37994,40.71569,-73.99076,-0.373600,3.0,10.0,0.36
37996,40.68591,-73.91344,-0.448553,1.0,18.0,0.41
37998,40.83555,-73.87570,-0.531833,3.0,38.0,1.82


In [35]:
b = df.select_dtypes(include=['object'])
lis = b.columns

for i in lis:
    b[i] = labelencoder.fit_transform(b[i])
b

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,name,host_name,neighbourhood_group,neighbourhood,room_type,last_review
0,3624,1427,1,41,0,1502
1,11413,6107,1,212,1,1502
2,26563,5341,2,129,1,1311
3,10760,7080,2,93,2,1609
5,12268,3096,2,93,0,631
...,...,...,...,...,...,...
37993,26649,5334,1,84,0,1677
37994,1923,2226,2,35,1,900
37996,15405,566,1,28,1,1195
37998,15335,1937,0,150,1,1686


In [36]:
df = pd.concat([a,b], axis = 1)
df

,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,name,host_name,neighbourhood_group,neighbourhood,room_type,last_review
0,40.68533,-73.96808,-0.015493,2.0,20.0,0.24,3624,1427,1,41,0,1502
1,40.70492,-73.93341,-0.406912,1.0,25.0,0.96,11413,6107,1,212,1,1502
2,40.81064,-73.95734,-0.011329,14.0,6.0,0.41,26563,5341,2,129,1,1311
3,40.82665,-73.94669,-0.431897,2.0,52.0,1.95,10760,7080,2,93,2,1609
5,40.81251,-73.94524,-0.240351,1.0,2.0,0.06,12268,3096,2,93,0,631
...,...,...,...,...,...,...,...,...,...,...,...,...
37993,40.66892,-73.99136,0.051131,3.0,NaN,3.70,26649,5334,1,84,0,1677
37994,40.71569,-73.99076,-0.373600,3.0,10.0,0.36,1923,2226,2,35,1,900
37996,40.68591,-73.91344,-0.448553,1.0,18.0,0.41,15405,566,1,28,1,1195
37998,40.83555,-73.87570,-0.531833,3.0,38.0,1.82,15335,1937,0,150,1,1686


In [37]:
df = df.dropna()

y = df.pop('price')
X_train, X_test, y_train, y_test = train_test_split(df, y,
                                                    test_size=0.33, random_state=42)

In [38]:
# svr.fit(X_train, y_train)

# score(y_train, svr.predict(X_train))

In [44]:
xgbreg = XGB_train_score(X_train, y_train)
score(y_test, xgbreg.predict(X_test))
# XGB_train_score(X_test, y_test)

/opt/conda/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


---
R2,  0.997529207145057
mse,  0.00033000083985259616
mae,  0.012282115167590996
rmse,  0.018165925240752154
---
---
R2,  0.40468347342825384
mse,  0.07398996152861939
mae,  0.1901728742662373
rmse,  0.27201095847156487
---


In [45]:
xgbreg.feature_importances_

array([0.03050613, 0.04775161, 0.03358996, 0.02545964, 0.02581939,
       0.03125326, 0.02549286, 0.02100731, 0.0317568 , 0.69983804,
       0.02752501], dtype=float32)

In [43]:
RFreg = RF_train_score(X_train, y_train)
score(y_test, RFreg.predict(X_test))


---
R2,  0.4355133665223023
mse,  0.07539323369846067
mae,  0.1828956911640082
rmse,  0.27457828337008133
---
---
R2,  0.44441734673636835
mse,  0.06905156720185596
mae,  0.17750227753359385
rmse,  0.26277664888999547
---


In [46]:
RFreg.feature_importances_

array([6.42881810e-02, 1.49489300e-01, 1.01031590e-02, 2.90641644e-05,
       1.33475445e-02, 1.97529971e-03, 2.37411783e-04, 3.36455273e-05,
       1.32289672e-02, 7.42967162e-01, 4.30026440e-03])